In [1]:
! pip install accelerate transformers bitsandbytes peft datasets py7zr

In [2]:
!pip install trl py7zr auto-gptq optimum

In [3]:
from datasets import load_dataset, Dataset

In [4]:
df = load_dataset('samsum', split='train')
df

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [5]:
data_df = df.to_pandas()
data_df.head(2)

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...


In [6]:
data_df.shape

(14732, 3)

In [7]:
data = data_df.sample(50)
data.shape

(50, 3)

In [8]:
data.head()

,id,dialogue,summary
10471,13727938,Timmy: Bt aren't the guest lists like closed a...,Guest list is never closed. Gemma knows that b...
7846,13681416,Simon: did the couch already came?\r\nZara: no...,Simon gets impatient as the couch hasn't been ...
5479,13727922,Sophie: <file_photo>\r\nAva: You look so happy...,Sophie is in Kutno. Ava is on her way to Bruss...
3780,13680321,Ella: Good morning Any. Sorry to trouble you a...,Andy has left the fridge door open at Ella's r...
13487,13810154,Andy: Hi mom! Is everything ok at home?\r\nSus...,Susan wants Andy to come home next weekend and...


In [9]:
data['text'] = data[["dialogue", "summary"]].apply(lambda x: "###Human: Summarize this following dialogue: " + x["dialogue"] + "\n###Assistant: " + x["summary"], axis=1)

In [10]:
data.head()

,id,dialogue,summary,text
10471,13727938,Timmy: Bt aren't the guest lists like closed a...,Guest list is never closed. Gemma knows that b...,###Human: Summarize this following dialogue: T...
7846,13681416,Simon: did the couch already came?\r\nZara: no...,Simon gets impatient as the couch hasn't been ...,###Human: Summarize this following dialogue: S...
5479,13727922,Sophie: <file_photo>\r\nAva: You look so happy...,Sophie is in Kutno. Ava is on her way to Bruss...,###Human: Summarize this following dialogue: S...
3780,13680321,Ella: Good morning Any. Sorry to trouble you a...,Andy has left the fridge door open at Ella's r...,###Human: Summarize this following dialogue: E...
13487,13810154,Andy: Hi mom! Is everything ok at home?\r\nSus...,Susan wants Andy to come home next weekend and...,###Human: Summarize this following dialogue: A...


In [12]:
data = Dataset.from_pandas(data)

In [13]:
data

Dataset({
    features: ['id', 'dialogue', 'summary', 'text', '__index_level_0__'],
    num_rows: 50
})

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments

In [15]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")

In [16]:
tokenizer.eos_token

'</s>'

In [17]:
tokenizer.pad_token = tokenizer.eos_token

# Loading the quantized model (GPTQ

In [18]:
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [19]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                                             quantization_config=quantization_config_loading,
                                             device_map="auto")

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [20]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-05)
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [21]:
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()

In [22]:
from peft import prepare_model_for_kbit_training

In [23]:
model = prepare_model_for_kbit_training(model)

In [26]:
from peft import LoraConfig, get_peft_model

In [25]:
peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)

In [27]:
model = get_peft_model(model, peft_config)

In [28]:
from trl import SFTTrainer # Transformer Reinforcement Learning

In [29]:
from transformers import TrainingArguments

In [30]:
training_arguments = TrainingArguments(
    output_dir="mistral-finetuned-samsum",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=1,
    max_steps=250,
    fp16=True,
)

In [34]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    # dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    # packing=False,
    # max_seq_length=512
)

<ipython-input-34-d829205a8e3d>:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,1.183600


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/

In [ ]:
trainer.push_to_hub()

In [ ]:
! cp -r /content/mistral-finetuned-samsum /content/drive/MyDrive/

# Inferenceing

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/mistral-finetuned-samsum")

In [ ]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: sunny: I'm at the railway station in Chennai Karthik: No problems so far? Vasanth: no, everything's going smoothly Karthik: good. lets meet there soon!
###Assistant: """, return_tensors="pt").to("cuda")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-samsum",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time() - st_time)